# Imports

In [37]:
import pandas as pd
import seaborn as sb
import matplotlib
from matplotlib import cm
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

matplotlib.style.use('ggplot')
%matplotlib notebook
import mpld3
mpld3.enable_notebook()

## Retrieve Data

In [31]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

/Users/Jojo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Exploration

In [32]:
train_data.dtypes

id               int64
qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object

In [33]:
train_data.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


# Baseline model

In [34]:
# feature: words per question

def count_words(question):
    question = str(question)
    words = question.split()
    return len(words)

train_data["q1l"] = train_data["question1"].apply(count_words)
train_data["q2l"] = train_data["question2"].apply(count_words)
test_data["q1l"] = test_data["question1"].apply(count_words)
test_data["q2l"] = test_data["question2"].apply(count_words)

In [35]:
# feature: repeated words per question

def count_repeats(row):
    q1 = str(row['question1']).lower()
    q2 = str(row['question2']).lower()
    
    w1 = q1.split()
    w2 = q2.split()
    
    count = 0
    
    if len(w1) > len(w2):
        for word in w1:
            if word in w2:
                count = count + 1
    else:
        for word in w2:
            if word in w1:
                count = count + 1
    
    return count

train_data["repeats"] = train_data.apply(count_repeats, axis=1)
test_data["repeats"] = test_data.apply(count_repeats, axis=1)

##### Initial Test

In [41]:
tp = 0
tn = 0
fp = 0
fn = 0

for index, row in train_data.iterrows():
    if (row['repeats']/row['q1l'] + row['repeats']/row['q2l'])/2 >= 0.5:
        if row['is_duplicate'] == 1:
            tp = tp + 1
        else:
            fp = fp + 1
    else:
        if row['is_duplicate'] == 1:
            fn = fn + 1
        else:
            tn = tn + 1

Accuracy:  0.6695490860520913


In [42]:
print("Accuracy: ", (tp+tn)/(tp+tn+fp+fn))
print("False positives: ", fp)
print("False negatives: ", fn)
print("True positives: ", tp)
print("True negatives: ", tn)

Accuracy:  0.6695490860520913
False positives:  87292
False negatives:  46306
True positives:  102957
True negatives:  167735


### Ready train/validation

In [38]:
features = ["repeats", "q1l", "q2l"]

X = train_data[features]
y = train_data["is_duplicate"]

X_train, X_test, y_train, y_val = train_test_split(X, y, test_size = 0.33)

,repeats,q1l,q2l
0,12,14,12
1,5,8,13
2,4,14,10
3,0,11,9
4,2,13,7
5,8,16,16
6,0,4,11
7,4,7,9
8,6,8,8
9,3,9,9
